## DEV :: TOURETZKY DYNAMICS

In [ ]:
import tqdm
import numpy as np
import matplotlib.pyplot as plt

from model import NN
import ops, plot, eq

np.set_printoptions(linewidth=np.inf, suppress=True, precision=6)

---

### DEFINE MODEL

In [ ]:
N = 120
net = NN(N)

In [ ]:
n_opt = 2                                       # One superior, rest inferior
v = 0.2                                         # Superior value
delta = 1.0                                     # Ratio (inferior/superior)
mu0 = [v] + [v * delta] * (n_opt-1)             # Value array

net.X, pid = ops.create_N_targets(N, mu0, "gaussian", sigma=6.0, theta=80, offset=0, bypass=False)

print(f"Unit Index: {pid} \nAmplitude: {mu0} \nDegrees (°): {ops.n2deg(pid, N)}")

In [ ]:
z_hist, z0_hist = [], []

n_steps = 10000
for i in tqdm.tqdm(range(n_steps)):
    net.dynamics()

    z_hist.append(net.z)
    z0_hist.append(net.z0)

In [ ]:
time_ms = np.arange(n_steps) / 1000

plt.figure(figsize=(20, 20))

# Neural Activity
plt.subplot2grid((3, 3), (0, 0), colspan=2)
plot.neural_activity(N, n_steps, net.X, net.z, None)

# Centre of Mass (COM)
plt.subplot2grid((3, 3), (0, 2), projection='polar')
plot.com_trajectory(N, time_ms, z_hist, pid, mu0)

# Pooled Inhibition
plt.subplot2grid((3, 3), (1, 0), colspan=2)
plot.simple_xy(time_ms, z0_hist, "Pooled Inhibition", "Time (ms)", "Response")

# Time Slice
plt.subplot2grid((3, 3), (1, 2), projection='3d')
plot.time_evolution(N, time_ms, z_hist, n_samples=6)

# Activation Function
plt.subplot2grid((3, 3), (2, 0))
x_act = np.linspace(-5, 5, 100)
plot.simple_xy(x_act, eq.relu(x_act), "Activation Function", "Input", "Response")

# Weight Profile & Init Cond
plt.subplot2grid((3, 3), (2, 1))
plot.weight_profile(N, net.wij, z_hist[0])

# Connectivity Matrix
plt.subplot2grid((3, 3), (2, 2))
plot.matrix(net.wij, "Connectivity Matrix", "i", "j")

plt.tight_layout()